In [3]:
import numpy as np
import sklearn.preprocessing as prep
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data


In [4]:
def xavier_init(fan_in, fan_out, constant=1):
    low = -constant * np.sqrt(6.0 / (fan_in + fan_out))
    high = constant * np.sqrt(6.0 / (fan_in + fan_out))
    
    return tf.random_uniform((fan_in , fan_out), 
                              minval = low , maxval = high, 
                            dtype = tf.float32)

In [74]:
class AdditiveGaussianNoiseAutoencoder(object):
    
    def __init__(self, n_input, n_hidden, transfer_function=tf.nn.softplus, 
                 optimizer = tf.train.AdamOptimizer(), scale=0.1):
        
        self.n_input = n_input
        self.n_hidden = n_hidden 
        self.transfer = transfer_function
        self.scale = tf.placeholder(tf.float32)
        self.training_scale = scale 
        network_weights = self._initializer_weights()
        self.weights = network_weights
        
        self.x = tf.placeholder(tf.float32, [None, self.n_input])
        self.hidden = self.transfer(tf.add(tf.matmul(
                                        self.x + scale * tf.random_normal((n_input,)),
                                        self.weights['w1']), self.weights['b1']))
    
        self.reconstruction = tf.add(tf.matmul(self.hidden, 
                                                self.weights['w2']), self.weights['b2'])
        
        
        self.cost = 0.5 * tf.reduce_sum(tf.pow(tf.subtract(
                                    self.reconstruction, self.x), 2.0))
        self.optimizer = optimizer.minimize(self.cost)
        
        init = tf.global_variables_initializer()
        self.sess = tf.Session()
        self.sess.run(init)
        
    def _initializer_weights(self):
    
        all_weights = dict()
        all_weights['w1'] = tf.Variable(xavier_init(self.n_input, 
                                                    self.n_hidden))

        all_weights['b1'] = tf.Variable(tf.zeros([self.n_hidden], 
                                                    dtype = tf.float32))
        all_weights['w2'] = tf.Variable(tf.zeros([self.n_hidden, 
                                                    self.n_input], dtype = tf.float32))

        all_weights['b2'] = tf.Variable(tf.zeros([self.n_input], 
                                                 dtype = tf.float32))
        return all_weights
    
    def partial_fit(self, X):
        cost, opt = self.sess.run((self.cost, self.optimizer), 
                                     feed_dict = {self.x : X, self.scale: self.training_scale})
        return cost 
    
    def calc_total_cost(slef, X):

        return self.sess.run(self.cost, feed_dict = {self.x : X, 
                                                    self.scale: self.training_scale})
    def transform(self, X):

        return self.sess.run(self.hidden, feed_dict = {self.x : X, 
                                                          self.scale: self.training_scale})

    def generate(self, hidden = None):

        if hidden is None : 
            hidden = np.random.normal(size = self.weights['b1'])

        return self.sess.run(self.reconstruction, 
                                feed_dict = {self.hidden:hidden})

    def reconstruct(self, X):

        return self.sess.run(self.reconstruction, feed_dict = {self.x:X, 
                                                                  self.scale: self.training_scale})
    def getWeights(self):

        return self.sees.run(self.weights['w1'])

    def getBiases(self):

        return self.sess.run(self.weights['b1'])
    


In [21]:
mnist = input_data.read_data_sets('MNIST_data', one_hot = True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [ ]:
# standrize the data_set.. instead of using MaxMin , we use ...

In [66]:
def standard_scale(X_train, X_test):
    preprocessor = prep.StandardScaler().fit(X_train)
    X_Train = preprocessor.transform(X_train)
    X_test = preprocessor.transform(X_test)
    
    return X_train, X_test

In [67]:
def get_random_block_from_data(data, batch_size):
    
    start_index = np.random.randint(0, len(data) -batch_size)
    
    return data[start_index:(start_index + batch_size)]

In [84]:
n_samples = int(mnist.train.num_examples)
training_epochs = 20
batch_size = 128
display_step = 1

In [85]:
X_train, X_test = standard_scale(mnist.train.images, mnist.test.images)

In [86]:
autoencodeer = AdditiveGaussianNoiseAutoencoder(n_input = 784, 
                                               n_hidden = 200, 
                                               transfer_function = tf.nn.softplus, 
                                               optimizer = tf.train.AdamOptimizer(learning_rate = 0.001), 
                                               scale = 0.01)

In [88]:
for epoch in range(training_epochs):
    
    avg_cost = 0.
    total_batch = int(n_samples / batch_size)
    
    for i in range(total_batch):
        
        batch_xs = get_random_block_from_data(X_train, batch_size)
        
        cost = autoencodeer.partial_fit(batch_xs)
        avg_cost += cost / n_samples * batch_size
        
    if epoch % display_step == 0:
        
        print('Epoch: ', '%04d'%(epoch + 1), 'cost= ', 
              '{:.9f}'.format(avg_cost))
        

Epoch:  0001 cost=  1105.196905256
Epoch:  0002 cost=  375.803518111
Epoch:  0003 cost=  286.923969141
Epoch:  0004 cost=  246.333073438
Epoch:  0005 cost=  223.527925142
Epoch:  0006 cost=  207.205850781
Epoch:  0007 cost=  196.941118608
Epoch:  0008 cost=  188.429349751
Epoch:  0009 cost=  183.491486648
Epoch:  0010 cost=  179.916228764
Epoch:  0011 cost=  176.493121307
Epoch:  0012 cost=  175.436255966
Epoch:  0013 cost=  174.717839986
Epoch:  0014 cost=  173.342107884
Epoch:  0015 cost=  170.705293643
Epoch:  0016 cost=  172.880268466
Epoch:  0017 cost=  168.920769460
Epoch:  0018 cost=  168.358138068
Epoch:  0019 cost=  169.065333416
Epoch:  0020 cost=  168.022548260
